In [1]:
import pandas as pd

In [ ]:
def merge_emoji(train):
    emoji = pd.read_csv('../input/emoji-sentiment-data/Emoji_Sentiment_Data_v1.0.csv')
    emoji2 = pd.read_csv('../input/emoji-sentiment-data/Emojitracker_20150604.csv')
    emoji = emoji.merge(emoji2, how='left', on='Emoji', suffixes=('', '_tracker'))
    
    emoji_list = emoji['Emoji'].values
    train_emoji = train['Description'].apply(extract_emojis, emoji_list=emoji_list)
    train_emoji = pd.DataFrame([train['PetID'], train_emoji]).T.set_index('PetID')
    train_emoji = train_emoji['Description'].str.extractall('(' + ')|('.join(emoji_list) + ')')
    train_emoji = train_emoji.fillna(method='bfill', axis=1).iloc[:, 0].reset_index().rename(columns={0: 'Emoji'})
    train_emoji = train_emoji.merge(emoji, how='left', on='Emoji')
    
    emoji_columns = ['Occurrences', 'Position', 'Negative', 'Neutral', 'Positive', 'Occurrences_tracker']
    stats = ['mean', 'max', 'min', 'median', 'std']
    g = train_emoji.groupby('PetID')[emoji_columns].agg(stats)
    g.columns = [c + '_' + stat for c in emoji_columns for stat in stats]
    train = train.merge(g, how='left', on='PetID')
    
    return train

In [2]:
emoji = pd.read_csv('../input/emoji-sentiment-data/Emoji_Sentiment_Data_v1.0.csv')
emoji2 = pd.read_csv('../input/emoji-sentiment-data/Emojitracker_20150604.csv')
emoji = emoji.merge(emoji2, how='left', on='Emoji', suffixes=('', '_tracker'))

In [3]:
emoji

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block,Unicode codepoint_tracker,Occurrences_tracker
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons,1F602,754843296.0
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats,2764,341911962.0
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols,2665,524698612.0
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons,1F60D,319310348.0
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons,1F62D,235371370.0
5,😘,0x1f618,3648,0.854480,193,702,2753,FACE THROWING A KISS,Emoticons,1F618,218118200.0
6,😊,0x1f60a,3186,0.813302,189,754,2243,SMILING FACE WITH SMILING EYES,Emoticons,1F60A,239396341.0
7,👌,0x1f44c,2925,0.805223,274,728,1923,OK HAND SIGN,Miscellaneous Symbols and Pictographs,1F44C,216474343.0
8,💕,0x1f495,2400,0.765726,99,683,1618,TWO HEARTS,Miscellaneous Symbols and Pictographs,1F495,207761611.0
9,👏,0x1f44f,2336,0.787130,243,634,1459,CLAPPING HANDS SIGN,Miscellaneous Symbols and Pictographs,1F44F,58380838.0


In [9]:
def extract_emojis(text, emoji_list):
    return ' '.join(c for c in text if c in emoji_list)

emoji_list = emoji['Emoji'].values
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
train[['Description', 'Name']] = train[['Description', 'Name']].astype(str)
train_emoji = train['Description'].apply(extract_emojis, emoji_list=emoji_list)
train_emoji = pd.DataFrame([train['PetID'], train_emoji]).T.set_index('PetID')
train_emoji.head()

,Description
PetID,
86e1089a3,
6296e909a,
3422e4906,
5842f1ff5,
850a43f90,


In [13]:
train_emoji = train_emoji['Description'].str.extractall('(' + ')|('.join(emoji_list) + ')')
train_emoji.head()

0    1    2    3    4    5    6    7    8    9   ...  959  \
PetID     match                                                   ...        
b0dec8779 0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...  NaN   
          1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...  NaN   
1e62e6022 0      NaN  NaN    ♥  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...  NaN   
2ac4dd8c2 0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...  NaN   
          1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...  NaN   

                960 961 962  963 964 965 966 967  968  
PetID     match                                        
b0dec8779 0     NaN NaN NaN  NaN NaN NaN NaN NaN  NaN  
          1     NaN NaN NaN  NaN NaN NaN NaN NaN  NaN  
1e62e6022 0     NaN NaN NaN  NaN NaN NaN NaN NaN  NaN  
2ac4dd8c2 0     NaN NaN NaN  NaN NaN NaN NaN NaN  NaN  
          1     NaN NaN NaN  NaN NaN NaN NaN NaN  NaN  

[5 rows x 969 columns]

In [14]:
train_emoji = train_emoji.fillna(method='bfill', axis=1).iloc[:, 0].reset_index().rename(columns={0: 'Emoji'})
train_emoji.head()


,PetID,match,Emoji
0,b0dec8779,0,♦
1,b0dec8779,1,♦
2,1e62e6022,0,♥
3,2ac4dd8c2,0,☺
4,2ac4dd8c2,1,😀


In [16]:
train_emoji = train_emoji.merge(emoji, how='left', on='Emoji')
train_emoji.head()

,PetID,match,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block,Unicode codepoint_tracker,Occurrences_tracker
0,b0dec8779,0,♦,0x2666,61,0.562227,6,20,35,BLACK DIAMOND SUIT,Miscellaneous Symbols,2666,4638684.0
1,b0dec8779,1,♦,0x2666,61,0.562227,6,20,35,BLACK DIAMOND SUIT,Miscellaneous Symbols,2666,4638684.0
2,1e62e6022,0,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols,2665,524698612.0
3,2ac4dd8c2,0,☺,0x263a,2062,0.798634,128,449,1485,WHITE SMILING FACE,Miscellaneous Symbols,263A,218194544.0
4,2ac4dd8c2,1,😀,0x1f600,439,0.786285,37,114,288,GRINNING FACE,Emoticons,1F600,20779182.0
